# Form the Logistic Regression Random Intercept Model: Simulated set

In this notebook we will use simulated dataset using MixedModels.jl to test the fit of our copula model on the logistic regression outcome.

For a single observation, $i = 1$ we will use ForwardDiff.jl to check the following calculations:

    (1) Approximate Hessian

The second differential (Hessian) is
\begin{eqnarray*}
&  & \sum_{i=1}^n \sum_j d^2 \ln f_{ij}(y_{ij} \mid \mathbf{\beta}) 
+ \sum_{i=1}^n\frac{\nabla \mathbf{r_i}(\mathbf{\beta})\mathbf{\Gamma_i} d\mathbf{r_i}(\mathbf{\beta})}{1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})} \\
&  &  +\sum_{i=1}^n
\frac{d^2 \mathbf{r_i}(\mathbf{\beta})\mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})}{1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})} -\sum_{i=1}^n\frac{[\nabla \mathbf{r_i}(\mathbf{\beta})\mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})]
[\nabla \mathbf{r_i}(\mathbf{\beta})\mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})]^t}
{\Big[1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})\Big]^2}.
\end{eqnarray*}
In searching the likelihood surface, we want to avoid second
derivatives. We also want an approximate Hessian that is
negative definite.  This suggests replacing $d^2 \ln f_{ij}(y_{ij} \mid \mathbf{\beta})$ by the negative information matrix $-\mathbf{J_{ij}}(\mathbf{\beta})$
under the independence model and dropping the second two sums
in the Hessians. 

In the paper, we want to avoid computationally burdensome second derivatives and ensure negative semi-definiteness, which gives the approximate Hessian:

 \begin{eqnarray*}
& = &- \sum_{i=1}^n \sum_j \mathbf{J}_{ij}(\mathbf{\beta})
 -\sum_{i=1}^n\frac{[\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})]
[\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma}_i \mathbf{r}_i(\mathbf{\beta})]^t}
{\Big[1+\frac{1}{2}r_i(\mathbf{\beta})^t \mathbf{\Gamma}_i r_i(\mathbf{\beta})\Big]^2}\\
&=& - \sum_{i=1}^n \mathbf{X_i}^T \mathbf{W_{2i}} \mathbf{X_i} -\sum_{i=1}^n\frac{[\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})]
[\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})]^t}
{\Big[1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})\Big]^2}
\end{eqnarray*}


From implementing just the above as the approximate Hessian we see that the Ipopt solver takes ~40 iterations to converge, while the hessian_approximation option uses the exact hessian (with second order term) and converges after 22 iterations. To get a closer approximate Hessian, we will try adding the second sum (all but the second derivative term).

 \begin{eqnarray*}
& = &- \sum_{i=1}^n \sum_j \mathbf{J}_{ij}(\mathbf{\beta}) + \sum_{i=1}^n\frac{\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma_i}\nabla \mathbf{r_i}(\mathbf{\beta})}{1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})} 
 -\sum_{i=1}^n\frac{[\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})]
[\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma}_i \mathbf{r}_i(\mathbf{\beta})]^t}
{\Big[1+\frac{1}{2}r_i(\mathbf{\beta})^t \mathbf{\Gamma}_i r_i(\mathbf{\beta})\Big]^2}\\
&=& - \sum_{i=1}^n \mathbf{X_i}^T \mathbf{W_{2i}} \mathbf{X_i} + \sum_{i=1}^n\frac{\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma_i} \nabla \mathbf{r_i}(\mathbf{\beta})}{1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})} -\sum_{i=1}^n\frac{[\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})]
[\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})]^t}
{\Big[1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})\Big]^2}
\end{eqnarray*}


In [43]:
using DataFrames, MixedModels, Random, GeneralizedCopula, GLM
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1235)
p = 10   # number of genes

n = 100  # number of cells

df = DataFrame(gene = repeat('A':'J', outer=n), normal = rand(n * p), nresp = ones(n * p))

#lmm1 = lmm(@formula(nresp ~ 1 + (1|gene)), df);
lmm1 = LinearMixedModel(@formula(nresp ~ 1 +  normal + (1|gene)), df);
# simulate the linear mixed model response and fit it (as a sanity check)
refit!(simulate!(lmm1, β=[0.2, 1.5], σ=0.2, θ = [1.]))

# # #simulate the Logistic response
df[!, :counts] = rand.(Bernoulli.(GLM.linkinv.(canonicallink(Bernoulli()), response(lmm1))))
df

,gene,normal,nresp,counts
,Char,Float64,Float64,Bool
1,'A',0.0951588,1.0,0
2,'B',0.270898,1.0,0
3,'C',0.906315,1.0,0
4,'D',0.833585,1.0,1
5,'E',0.945055,1.0,1
6,'F',0.443669,1.0,1
7,'G',0.904577,1.0,1
8,'H',0.941598,1.0,0
9,'I',0.0375897,1.0,0


In [44]:
groups = unique(df[!, :gene])
n, p, m = length(groups), 1, 1
d = Bernoulli()
D = typeof(d)
gcs = Vector{GLMCopulaVCObs{Float64, D}}(undef, n)
for (i, grp) in enumerate(groups)
    gidx = df[!, :gene] .== grp
    ni = count(gidx)
    y = Float64.(df[gidx, :counts])
    normal = Float64.(df[gidx, :normal])
    X = [ones(ni, 1) normal]
    V = [ones(ni, ni)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d)
end
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
# gcm.β .= [0.5, 2]
@show gcm.β

1 0.0 -641.333341152059 39
2 -641.333341152059 -572.8863440551082 9
3 -572.8863440551082 -569.0832094974016 9
4 -569.0832094974016 -569.080574334758 9
5 -569.080574334758 -569.0805743267674 9
gcm.β = [0.12924041019880786, 1.7101020114656673]


2-element Array{Float64,1}:
 0.12924041019880786
 1.7101020114656673

## MM update for Sigma

$$
  \begin{eqnarray}
\mathcal{L}(\sigma) = - \sum_i^n \ln \left(1 + \sum_k^m \sigma_k^2 t_{ik}\right) + \sum_i^n \ln \left(1 + \frac{1}{2} \sum_k^m \sigma_k^2 q_{ik}\right)
\end{eqnarray}
$$

where
$$
\begin{eqnarray*}
	t_{ik} &=& \frac 12 \text{tr}(\mathbf{V}_{ik}), \quad \mathbf{t}_i = (t_{i1}, \ldots, t_{im})^T \\
	q_{ik} &=& \frac{1}{2}  \mathbf{r}_i( \boldsymbol{\beta})^T \mathbf{V}_{ik}  \mathbf{r}_i (\boldsymbol{\beta}), \quad \mathbf{q}_i = (q_{i1}, \ldots, q_{im})^T.
\end{eqnarray*}
$$

where the $k^{th}$ element in the $m \times 1$ vector of variance components above is:

$$\begin{eqnarray*}
\sigma_{k}^{2^{(t+1)}} & = & \sigma_{k}^{2^{(t)}} \left(\frac{\sum_{i=1}^n
\frac{q_{ik}}{1+ q_i^{(t)}}}{\sum_{i=1}^n \frac{t_{ik}}{1+ t_i^{(t)}}} \right).
\end{eqnarray*}
$$

where $q_i^{(t)} = \sum_{k=1}^m \sigma_k^{2(t)} q_{ik}$ and $t_i^{(t)} = \sum_{k=1}^m \sigma_k^{2(t)} t_{ik}$. 


In this case $m = 1$ single variance component to model the random intercept. 

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [28]:
fill!(gcm.Σ, 1.0)
update_Σ!(gcm)
GeneralizedCopula.loglikelihood3!(gcm, true, true)

-590.449886072176

Closer Look at Observation 1
i =1

In [4]:
gc = gcm.data[1]
β  = gcm.β
Σ  = gcm.Σ
τ  = gcm.τ

@show β
@show Σ
@show τ

n_i  = length(gc.y)

β = [0.5, 2.0]
Σ = [0.10367203044619394]
τ = [1.0]


100

## Term 1: Fisher's Expected Information


\begin{eqnarray*}
& = & \mathbf{FIM}_n(\beta) = \mathbf{E} [- \nabla^2 L_n(\beta)]\\ &=& - \sum_{i=1}^n \sum_j \mathbf{J}_{ij}(\mathbf{\beta}) = - \sum_{i=1}^n \sum_{j=1}^{n_i} \frac{[\mu_{ij}'(\eta_{ij})]^2}{\sigma_{ij}^2} \mathbf{x}_{ij} \mathbf{x}_{ij}^T = - \sum_{i=1}^n \mathbf{X_i}^T \mathbf{W_{2i}} \mathbf{X_i}.
\end{eqnarray*}

### First I want to check if the mean and residuals are updated and standardized at this point

In [5]:
@test gc.η == gc.X*β                        # systematic linear component
@test gc.μ == exp.(gc.η)./(1 .+ exp.(gc.η)) # mu = ginverse of XB = mean component for GLM = [p]
@test gc.varμ == gc.μ .*(1 .- gc.μ)         # variance of the Bernoulli response as a function of mean mu [p(1-p)]
@test gc.res ≈ (gc.y - gc.μ)./sqrt.(gc.varμ)# standardized residual for GLM [(y - p)/sqrt(p(1-p))]

Test Passed

## Group i = 1, j in [1, n_1]


$$\text{Term 1 }= - \sum_{j=1}^{n_1} \frac{[\mu_{1j}'(\eta_{1j})]^2}{\sigma_{1j}^2} \mathbf{x}_{1j} \mathbf{x}_{1j}^T = - \mathbf{X_1}^T \mathbf{W_{21}} \mathbf{X_1}. $$

In [6]:
# these are slightly off by small decimals
@test gc.dμ ≈ exp.(gc.η)./(1 .+ exp.(gc.η)).^2        # derivative of mean with respect to systematic component

# First check if the diagonal elements of the W21 matrix is equal to the theoretical weights
@test gc.w2 == (gc.dμ).^2 ./ gc.varμ

term1_hard_code = -transpose(gc.X)*Diagonal(gc.w2)*gc.X

2×2 Array{Float64,2}:
 -15.7599   -5.84941
  -5.84941  -3.25041

In [7]:
# term 1:
"""
glm_hessian(gc)
Compute the part of the hessian relevant to the glm density with respect to beta for a single obs
"""
function hessian_glm(
    gc::Union{GLMCopulaVCObs{T, D}, GaussianCopulaVCObs{T, D}},
    β) where {T <: BlasReal, D}
    score = zeros(size(gc.∇β))
    inform = zeros(size(gc.∇β, 1), size(gc.∇β, 1))
    update_res!(gc, β)
        for j in 1:length(gc.y)
              c = gc.dμ[j]^2 / gc.varμ[j]
              BLAS.ger!(-c, gc.X[j, :], gc.X[j, :], inform) # inform = inform + c * x * x'
        end
   inform
end

term1 = hessian_glm(gc, β)

2×2 Array{Float64,2}:
 -15.7599   -5.84941
  -5.84941  -3.25041

###  Test if the hessian of the component loglikelihood matches our hessian function

In [8]:
function logistic_density2(β::Vector)
    η = gc.X*β                        # systematic linear component
    μ = exp.(η)./(1 .+ exp.(η)) # mu = ginverse of XB = mean component for GLM = [p]
    dμ = exp.(η)./(1 .+ exp.(η)).^2
    varμ = dμ
    logl = sum(gc.y .* log.(μ) .+ (1 .- gc.y).*log.(1 .- μ))
end

h1 = x -> ForwardDiff.hessian(logistic_density2, x)

hessian1magictest = h1(β)
@show hessian1magictest

hessian1magictest = [-15.759940823284133 -5.8494097196481745; -5.8494097196481745 -3.2504102210573786]


2×2 Array{Float64,2}:
 -15.7599   -5.84941
  -5.84941  -3.25041

## Term 2: Copula Model Specific

The hessian of our model specific component is the partial of this second term in the gradient:


$$\begin{eqnarray*}
\nabla_\beta \text{Term 2} &=& \sum_{i=1}^n
\frac{\nabla \mathbf{r_i(\mathbf{\beta})}\mathbf{\Gamma_i}\mathbf{r_i(\mathbf{\beta})}}{1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i(\mathbf{\beta})}}
\end{eqnarray*}
$$

$$ H_\beta \text{Term 2} = - \frac{[\nabla \mathbf{r_1}(\mathbf{\beta})^t\mathbf{\Gamma_1} \mathbf{r_1}(\mathbf{\beta})]
[\nabla \mathbf{r_1}(\mathbf{\beta})^t\mathbf{\Gamma_1} \mathbf{r_1}(\mathbf{\beta})]^t}
{\Big[1+\frac{1}{2}\mathbf{r_1}(\mathbf{\beta})^t \mathbf{\Gamma_1} \mathbf{r_1}(\mathbf{\beta})\Big]^2}$$

We notice the quantity we need to first form this matrix: 

\begin{eqnarray*}
&  & \nabla \mathbf{r_1}(\mathbf{\beta})^t\mathbf{\Gamma_1} \mathbf{r_1}(\mathbf{\beta}) =
\sum_{k=1}^m \mathbf\Sigma[k] * \nabla \mathbf{r_1}(\mathbf{\beta})^t * \mathbf{V_1}[k] * \mathbf{r_1}(\mathbf{\beta})
\end{eqnarray*}

In [9]:
Γ1 = Σ[1]*gc.V[1]

hess_t2_numerator = transpose(gc.∇resβ) * Γ1 * gc.res       # new term ∇resβ^t * Γ * res
@show hess_t2_numerator

quadratic_form_half = (transpose(gc.res) * Γ1 * gc.res)/2
@show quadratic_form_half 

hess_t2_denominator = (1 + quadratic_form_half)^2
@show hess_t2_denominator

term2_hard_code = -(hess_t2_numerator * transpose(hess_t2_numerator)) / hess_t2_denominator

hess_t2_numerator = [58.40098289607255, 27.812298858051147]
quadratic_form_half = 6.385220974458291
hess_t2_denominator = 54.541488841578676


2×2 Array{Float64,2}:
 -62.5336  -29.7804
 -29.7804  -14.1823

In [10]:
function hessian_copula_addendum(
    gc::Union{GLMCopulaVCObs{T, D}, GaussianCopulaVCObs{T, D}},
    β::Vector{T},
    τ::T,
    Σ::Vector{T}
    ) where {T <: BlasReal, D}
    n, p, m = size(gc.X, 1), size(gc.X, 2), length(gc.V)
    update_res!(gc, β)
    if gc.d  ==  Normal()
        sqrtτ = sqrt.(τ[1])
        standardize_res!(gc, sqrtτ)
    else
        sqrtτ = 1.0
        standardize_res!(gc)
    end
    fill!(gc.∇β, 0.0) # fill gradient with 0
    fill!(gc.Hβ, 0) # fill hessian with 0
    fill!(gc.∇resβ, 0.0) # fill gradient of residual vector with 0
    std_res_differential!(gc) # this will compute ∇resβ
    tsum = dot(Σ, gc.t)
    for k in 1:m
        mul!(gc.storage_n, gc.V[k], gc.res) # storage_n = V[k] * res
        BLAS.gemv!('T', Σ[k], gc.∇resβ, gc.storage_n, 1.0, gc.∇β) # stores ∇resβ*Γ*res (standardized residual)
        gc.q[k] = dot(gc.res, gc.storage_n) / 2
    end
    #@show gc.∇β
    @show gc.q
    qsum  = dot(Σ, gc.q)
    @show qsum
        inv1pq = inv(1 + qsum)
    @show inv1pq
        BLAS.syrk!('L', 'N', -abs2(inv1pq), gc.∇β, 1.0, gc.Hβ) # only lower triangular
    
    added_term_numerator = zeros(n, p)
    added_term2 = zeros(p, p)
    for k in 1:m
        mul!(added_term_numerator, gc.V[k], gc.∇resβ) # storage_n = V[k] * res
        BLAS.gemm!('T', 'N', Σ[k], gc.∇resβ, added_term_numerator, 1.0, added_term2)
    end
    @show added_term2
    added_term2 .*= inv1pq
    @show added_term2
    gc.Hβ .+= added_term2
    @show gc.Hβ
end

hess_term2 = hessian_copula_addendum(gc, β, τ[1], Σ)

gc.q = [520.0359251065521]
qsum = 53.9131802607611
inv1pq = 0.018210564298978736
added_term2 = [267.0757062966572 127.18945800049637; 127.18945800049627 60.571432912325875]
added_term2 = [4.863599322210436 2.3161918030702946; 2.316191803070293 1.1030399737311871]
gc.Hβ = [-4.686461545847272 2.3161918030702946; -2.231833483553137 -1.062866012999236]


2×2 Array{Float64,2}:
 -4.68646   2.31619
 -2.23183  -1.06287

## Now I will check the part of the Loglikelihood that is specific to our density

I note that this portion given by ForwardDiff will be the exact hessian while I am using the Expected Information matrix to approximate it

In [11]:
function copula_loglikelihood_addendum1(β::Vector)
  m = length(gc.V)
  η = gc.X*β                        # systematic linear component
  μ = exp.(η)./(1 .+ exp.(η)) # mu = ginverse of XB = mean component for GLM = [p]
  varμ = exp.(η)./(1 .+ exp.(η)).^2
  res = (gc.y .- μ) ./ sqrt.(varμ)
  trace_gamma = Σ[1]*tr(gc.V[1])
  trace_gamma_half = trace_gamma/2

  term1 = -log(1 + trace_gamma_half) # -1.252762968495368
  quad_form_standardized_res_half = (Σ[1]*transpose(res)*gc.V[1]*res)/2
  term2 = log(1 + quad_form_standardized_res_half) # 0.0381700599136237
  logl_hard_coded_obs1 = term1 + term2
  logl_hard_coded_obs1
end

h2 = x -> ForwardDiff.hessian(copula_loglikelihood_addendum1, x)
hessian2magictest = h2(β)
@show hessian2magictest

hessian2magictest = [-4.195566827996745 -1.9002608288531575; -1.9002608288531584 -0.8257624141594377]


2×2 Array{Float64,2}:
 -4.19557  -1.90026
 -1.90026  -0.825762

## Added term to see if we can improve Hessian Approximation

$$
\sum_{i=1}^n\frac{\nabla \mathbf{r_i}(\mathbf{\beta})^t\mathbf{\Gamma_i}\nabla \mathbf{r_i}(\mathbf{\beta})}{1+\frac{1}{2}\mathbf{r_i}(\mathbf{\beta})^t \mathbf{\Gamma_i} \mathbf{r_i}(\mathbf{\beta})} 
$$

$i = 1$

$$
\text{Term Added: }\frac{\nabla \mathbf{r_1}(\mathbf{\beta})^t\mathbf{\Gamma_1}\nabla \mathbf{r_1}(\mathbf{\beta})}{1+\frac{1}{2}\mathbf{r_1}(\mathbf{\beta})^t \mathbf{\Gamma_1} \mathbf{r_1}(\mathbf{\beta})} 
$$




In [12]:
Γ1 = Σ[1]*gc.V[1]

added_term_numerator =  transpose(gc.∇resβ) * Γ1 * gc.∇resβ
@show added_term_numerator

quadratic_form_half = (transpose(gc.res) * Γ1 * gc.res)/2
@show quadratic_form_half 

added_term_denominator = inv(1 + quadratic_form_half)
@show added_term_denominator

added_term_hard_code = added_term_numerator * added_term_denominator

added_term_numerator = [267.07570629665724 127.18945800049639; 127.18945800049637 60.571432912325896]
quadratic_form_half = 53.91318026076111
added_term_denominator = 0.018210564298978733


2×2 Array{Float64,2}:
 4.8636   2.31619
 2.31619  1.10304

In [13]:
hessian_approximation_closer = term1_hard_code + term2_hard_code + added_term_hard_code

2×2 Array{Float64,2}:
 -73.4299  -33.3136
 -33.3136  -16.3297

## Now I will put together both the parts of the loglikelihood and both the parts of the gradient to check alltogether now

In [14]:
function full_loglikelihood(β::Vector)
    logl = 0.0
    logl = logistic_density2(β) + copula_loglikelihood_addendum1(β)
    logl
end

h = x -> ForwardDiff.hessian(full_loglikelihood, x)
hessianmagictest = h(β)
@show hessianmagictest

hessianmagictest = [-19.95550765128088 -7.7496705485013315; -7.749670548501333 -4.076172635216817]


2×2 Array{Float64,2}:
 -19.9555   -7.74967
  -7.74967  -4.07617

In [46]:
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
#gcm.β .= [0.5, 2]
@show gcm.β
fill!(gcm.Σ, 1.0)
update_Σ!(gcm)
GeneralizedCopula.loglikelihood3!(gcm, true, true)
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 100, derivative_test = "first-order", hessian_approximation = "limited-memory"))

1 0.0 -641.333341152059 39
2 -641.333341152059 -572.8863440551082 9
3 -572.8863440551082 -569.0832094974016 9
4 -569.0832094974016 -569.080574334758 9
5 -569.080574334758 -569.0805743267674 9
gcm.β = [0.12924041019880786, 1.7101020114656673]
This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Starting derivative checker for first derivatives.

* grad_f[          1] = -7.0981539601041595e+02    ~ -2.1850205635478569e+03  [ 6.751e-01]
* grad_f[          2] = -3.8909486350194783e+02    ~ -1.5388880092359614e+04  [ 9.747e-01]

Derivative checker detected 2 error(s).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        0
                va

GLMCopulaVCModel{Float64,Bernoulli{Float64}}(GLMCopulaVCObs{Float64,Bernoulli{Float64}}[GLMCopulaVCObs{Float64,Bernoulli{Float64}}([0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0  …  1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0], [1.0 0.09515880533766441; 1.0 0.23881672289989253; … ; 1.0 0.2707677310894794; 1.0 0.9916847794622285], [[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]], [-2.9072210102233673, -3.05711160153184], [-0.5785149551503376 -0.05505079200207863; -0.6542570098779004 -0.1562475150333228; … ; -0.6724076000387136 -0.1820662802298046; -0.2005215884796763 -0.1988542072488835], [1.5e-323], [2.2104556125e-314], [-20.212526732386472 -8.043109601563785; -8.193503370590694 -4.780236515971943], [4.243991583e-314], [-1.1570299103006751, -1.3085140197558007, 0.8468374293820895, 0.40454099875314353, -1.8019923483346916, -1.4805486648578887, 0.8561289384644659, 0.4053751066993921, -1.3671189645669357, 0.9235009662956577  …  0.8954110905167303,

In [30]:
@show loglikelihood3!(gcm, true, true)
@show gcm.β
@show gcm.∇β

loglikelihood3!(gcm, true, true) = -589.5715418459318
gcm.β = [0.030901807865707533, 1.4217389732846137]
gcm.∇β = [-1.5974777056726452e-9, -7.889777720038182e-10]


2-element Array{Float64,1}:
 -1.5974777056726452e-9
 -7.889777720038182e-10

In [45]:
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
#gcm.β .= [0.5, 2]
@show gcm.β
fill!(gcm.Σ, 1.0)
update_Σ!(gcm)
GeneralizedCopula.loglikelihood3!(gcm, true, true)
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 100, hessian_approximation = "exact"))

1 0.0 -641.333341152059 39
2 -641.333341152059 -572.8863440551082 9
3 -572.8863440551082 -569.0832094974016 9
4 -569.0832094974016 -569.080574334758 9
5 -569.080574334758 -569.0805743267674 9
gcm.β = [0.12924041019880786, 1.7101020114656673]
This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with on

GLMCopulaVCModel{Float64,Bernoulli{Float64}}(GLMCopulaVCObs{Float64,Bernoulli{Float64}}[GLMCopulaVCObs{Float64,Bernoulli{Float64}}([0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0  …  1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0], [1.0 0.09515880533766441; 1.0 0.23881672289989253; … ; 1.0 0.2707677310894794; 1.0 0.9916847794622285], [[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]], [-2.9072210496111675, -3.0571116187894845], [-0.5785149551785187 -0.05505079200476031; -0.6542570099006798 -0.15624751503876289; … ; -0.6724076000600465 -0.18206628023558086; -0.20052158848729773 -0.1988542072564416], [1.5e-323], [2.2104556125e-314], [-20.204497944536577 -8.037777246526328; -8.18783589645397 -4.775935948564466], [4.243991583e-314], [-1.1570299103570372, -1.3085140198013594, 0.8468374293427882, 0.4045409987681225, -1.8019923483323046, -1.480548664888782, 0.8561289384236779, 0.40537510671430765, -1.3671189646077688, 0.9235009662437592  …  0.89541109046953

In [32]:
@show loglikelihood3!(gcm, true, true)
@show gcm.β
@show gcm.∇β

loglikelihood3!(gcm, true, true) = -589.5715418457637
gcm.β = [0.030901807876090592, 1.4217389732904058]
gcm.∇β = [-7.2586647803518645e-9, -3.6487692867126498e-9]


2-element Array{Float64,1}:
 -7.2586647803518645e-9
 -3.6487692867126498e-9